# Evaluating the Model

This notebook follows training the model and will give a the final training and test error for the model. To download the trained model, follow the instructions in [README.md](README.md).

In [3]:
# Imports
import tensorboard as tb

In order to import that evaluations as a dataframe, the easiest way we could find was to upload the data to tensorboard.dev. It is located here: [Tensorboard.dev](https://tensorboard.dev/experiment/dTD0vaI3SdyRZYI4WLHRbg/#scalars). The following cell collects that data and turns it into a pandas dataframe with the run as the index and the various losses the the columns.

The documentation for this process can be found [here](https://www.tensorflow.org/tensorboard/dataframe_api)

In [31]:
experiment_id = "dTD0vaI3SdyRZYI4WLHRbg" # Located when data was uploaded to tensorboard
experiment = tb.data.experimental.ExperimentFromDev(experiment_id) # Download the data from the tensorboard project
df = experiment.get_scalars()
output  = df.pivot(index="run", columns="tag", values="value")
output = output.drop('eval')

In [32]:
output

tag,DetectionBoxes_Precision/mAP,DetectionBoxes_Precision/mAP (large),DetectionBoxes_Precision/mAP (medium),DetectionBoxes_Precision/mAP (small),DetectionBoxes_Precision/mAP@.50IOU,DetectionBoxes_Precision/mAP@.75IOU,DetectionBoxes_Recall/AR@1,DetectionBoxes_Recall/AR@10,DetectionBoxes_Recall/AR@100,DetectionBoxes_Recall/AR@100 (large),DetectionBoxes_Recall/AR@100 (medium),DetectionBoxes_Recall/AR@100 (small),Loss/classification_loss,Loss/localization_loss,Loss/regularization_loss,Loss/total_loss
run,,,,,,,,,,,,,,,,
test,0.577580,0.804416,0.420162,0.004719,0.823290,0.649656,0.369214,0.635027,0.658216,0.844016,0.557507,0.157143,0.223413,0.003314,0.037377,0.264104
training,0.600287,0.818538,0.414392,0.005160,0.840275,0.676805,0.393724,0.652730,0.672312,0.853080,0.558387,0.147729,0.181020,0.003023,0.037377,0.221419
validation,0.582718,0.802349,0.398904,0.007373,0.826129,0.655632,0.381782,0.640276,0.662531,0.840806,0.551840,0.156835,0.197278,0.003231,0.037377,0.237886


An interesting thing to note here is that the cost evaluated during the training was calculated on a subset of the whole training data set while this training error was calculated on the entire data set. Despite this the values are very similiar.

In general, the bounding box precision and recall, as well as the total loss are all relatively close together when evaluated on each of the three sets. The total loss had the highest change between training and test, most likely due to it being the combination of 4 different loss metrics.